# 네이버 영화 정보 크롤링
 - 1. csv파일에 500개의 영화명을 리스트화
 - 2. 영화명을 네이버 영화 검색창에 입력 
 - 3. 검색결과가 중복으로 여러개의 영화가 나타날 수 있고 원하는 영화는 몇번째 위치했는지 알 수 없어 첫번째 부터 4번째까지의 영화들을 클릭>> 사이트 이동
 - 4. 이동 된 사이트에 나와있는 평점, 제목, 개봉일 등등 데이터 수집
  - 이 과정에서 비주류 영화에는 평점, 제목 개봉일 등 각각의 데이터들이 없는 경우가 발견 오류로 이어짐 >> try로 None 처리 함
  - 모든 사이트를 분석한 것이 아니라 어떤 경우에 None 처리가 됐는지 불명확함 

In [ ]:
import requests
import pandas as pd
from urllib.parse import quote
from bs4 import BeautifulSoup
import re
import requests
import pandas as pd
from urllib.parse import quote
import unicodedata
from tqdm.notebook import tqdm
import numpy as np


In [ ]:
### 한글 숫자 필터링 함수
def test(s):
  hangul = re.compile('[^ ㄱ-ㅣ가-힣0-9+]') # 한글과 띄어쓰기를 제외한 모든 글자 # hangul = re.compile('[^ \u3131-\u3163\uac00-\ud7a3]+') # 위와 동일 
  result = hangul.sub('', s) # 한글과 띄어쓰기를 제외한 모든 부분을 제거 
  return (result)


In [ ]:
def test2(s):
  hangul = re.compile('[^ 0-9+.]')
  result = hangul.sub('', s) 
  return (result)

In [ ]:
name = '명량'    # 특정 예를 바탕으로 사이트의 규칙을 찾아보려합니다
url = f'https://movie.naver.com/movie/search/result.naver?query={name}&section=all'      # point 사이트 내에서 평점 , point 부분이 바뀌면 그 밖에 리뷰, 주요정보 등 다른 정보가 화면내 표시
req = requests.get(url)
html = req.text
#html

In [ ]:
soup =BeautifulSoup(html, 'html.parser')

In [ ]:
####################### 일반화 과정 #############
##### 영화 제목
name = '명량' #### 영화제목입력
url = f'https://movie.naver.com/movie/search/result.naver?query={name}&section=all'
req = requests.get(url)
html = req.text
soup =BeautifulSoup(html, 'html.parser') ## 영화제목 검색된 창
title_list =[]
star_list = []
jangru_list = []
country_list = []
time_list =[]
date_list = []
info_list = []
#for i in range(1,5):
i = 1
site_num = soup.select_one(f'#old_content > ul.search_list_1 > li:nth-of-type({i}) > dl > dt > a')      ### 중복되는 영화들 중 순서대로 4개만 크롤링
  ######### 영화 정보 사이트로 이동
#if site_num != None:
new_name = site_num['href']     # 영화 사이트 주소 가져오기
new_url = f'https://movie.naver.com/{new_name}'
new_url = new_url.replace('basic','point')      # 사이트에서 평점을 클릭 해야함
new_req = requests.get(new_url)
new_html = new_req.text
s =BeautifulSoup(new_html, 'html.parser')
    ##### 정보 가져오기 ########
title = s.select_one('#content > div.article > div.mv_info_area > div.mv_info > h3 > a')
title = title.text     ### 영화 제목
inf_m = s.select_one("#netizen_point_graph > div > strong")
inf = inf_m.text       ### 네이버 영화측에서 분석한 성별 연령층 정보 + 영화의 특징
star = s.select_one("div.main_score div.score a div.star_score span.st_off span.st_on")
star = star.text       ### 관람객 평점
date = s.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd > p')
date = date.text
m = date.split('\n\n') ## \n\n으로 구분 돼 장르, 나라, 시간, 개봉일이 표시되는 것 같습니다.
jangru = test(m[1])
country = test(m[2])
time = test(m[3])
date = test(m[4])
title_list.append(title)
star_list.append(star)
jangru_list.append(jangru)
country_list.append(country)
time_list.append(time)
date_list.append(date)
info_list.append(inf)





In [ ]:
############ 영화 제목을 리스트로 가져 오겠습니다 
#######
######
df = pd.read_csv('ever_kobis.csv')
name = df['영화명'].values
names = name.tolist()
names

['명량',
 '극한직업',
 '신과함께-죄와 벌',
 '국제시장',
 '어벤져스: 엔드게임',
 '겨울왕국 2',
 '아바타',
 '베테랑',
 '괴물',
 '도둑들',
 '7번방의 선물',
 '암살',
 '알라딘',
 '광해, 왕이 된 남자',
 '왕의 남자',
 '신과함께-인과 연',
 '택시운전사',
 '태극기 휘날리며',
 '부산행',
 '해운대',
 '변호인',
 '어벤져스: 인피니티 워',
 '실미도',
 '어벤져스: 에이지 오브 울트론',
 '기생충',
 '겨울왕국',
 '인터스텔라',
 '보헤미안 랩소디',
 '검사외전',
 '엑시트',
 '설국열차',
 '관상',
 '아이언맨 3',
 '캡틴 아메리카: 시빌 워',
 '해적: 바다로 간 산적',
 '수상한 그녀',
 '국가대표',
 '디워',
 '백두산',
 '과속스캔들',
 '스파이더맨: 파 프롬 홈',
 '웰컴 투 동막골',
 '공조',
 '트랜스포머 3',
 '히말라야',
 '미션임파서블:고스트프로토콜',
 '트랜스포머: 패자의 역습',
 '밀정',
 '최종병기 활',
 '트랜스포머',
 '써니',
 '화려한 휴가',
 '스파이더맨: 홈 커밍',
 '1987',
 '베를린',
 '마스터',
 '터널',
 '어벤져스',
 '내부자들',
 '인천상륙작전',
 '럭키',
 '은밀하게 위대하게',
 '곡성',
 '범죄도시',
 '타짜',
 '좋은 놈, 나쁜 놈, 이상한 놈',
 '늑대소년',
 '미녀는 괴로워',
 '군함도',
 '미션 임파서블: 폴아웃',
 '다크 나이트 라이즈',
 '아저씨',
 '사도',
 '전우치',
 '킹스맨 : 시크릿 에이전트',
 '미션 임파서블: 로그네이션',
 '투사부일체',
 '연평해전',
 '반지의 제왕 : 왕의 귀환',
 '인셉션',
 '레미제라블',
 '쉬리',
 '캡틴 마블',
 '미션 임파서블 3',
 '쥬라기 월드: 폴른 킹덤',
 '청년경찰',
 '가문의 위기(가문의 영광2)',
 '숨바꼭질',
 '덕혜옹주',
 '더 테러 라이브',
 '쥬라기 월드

In [ ]:
###### 일반화 된 크롤링 반복
#######
title_list =[]
star_list = []
jangru_list = []
country_list = []
time_list =[]
date_list = []
info_list = []
age1_list = []
age2_list = []
age3_list = []
age4_list = []
age5_list = []
man_list = []
woman_list = []
star2_list = []
story_list =[]
for name in names:
  url = f'https://movie.naver.com/movie/search/result.naver?query={name}&section=all'
  req = requests.get(url)
  html = req.text
  soup =BeautifulSoup(html, 'html.parser') ## 영화제목 검색된 창
  for i in range(1,6):
    site_num = soup.select_one(f'#old_content > ul.search_list_1 > li:nth-of-type({i}) > dl > dt > a')      ### 중복되는 영화들 중 순서대로 4개만 크롤링
    ######### 영화 정보 사이트로 이동
    if site_num != None:
      new_name = site_num['href']     # 영화 사이트 주소 가져오기
      new_url = f'https://movie.naver.com/{new_name}'
      new_url = new_url.replace('basic','point')      # 사이트에서 평점을 클릭 해야함
      new_req = requests.get(new_url)
      new_html = new_req.text
      s =BeautifulSoup(new_html, 'html.parser')
      ##### 정보 가져오기 ########
      title = s.select_one('#content > div.article > div.mv_info_area > div.mv_info > h3 > a')
      try:
        title = title.text     ### 영화 제목
      except:
        title = None
      inf_m = s.select_one("#netizen_point_graph > div > strong")
      try:
        inf = inf_m.text       ### 네이버 영화측에서 분석한 성별 연령층 정보 + 영화의 특징
      except:
        inf = None
      star = s.select_one("div.main_score div.score a div.star_score span.st_off span.st_on")
      try:
        star = star.text       ### 관람객 평점
      except:
        star = None
      star2 = s.select_one("#pointNetizenPersentBasic")
      try:
        star2 = round(float(test2(str(star2)).split()[1])/10,2)       ### 네티즌 평점
      except:
        star2 = None
      date = s.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd > p')
      try:
        date = date.text
      except:
        date = None
      try:
        m = date.split('\n\n') ## \n\n으로 구분 돼 장르, 나라, 시간, 개봉일이 표시되는 것 같습니다.
      except:
        m = [None, None, None, None]
      try:
        jangru = test(m[1])
      except:
        jangru = None
      try:
        country = test(m[2])
      except:
        country = None
      try:
        time = test(m[3])
      except:
        time = None
      try:
        date = test(m[4])
      except:
        date = None
      age1 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(1) > strong.graph_percent")
      try:
        age1 = age1.text
      except:
        age1 = 'x'
      age2 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(2) > strong.graph_percent")
      try:
        age2 = age2.text
      except:
        age2 = 'x'
      age3 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(3) > strong.graph_percent")
      try:
        age3 = age3.text
      except:
        age3 = 'x' 
      age4 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(4) > strong.graph_percent")
      try:
        age4 = age4.text
      except:
        age4 = 'x'   
      age5 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(5) > strong.graph_percent")
      try:
        age5 = age5.text
      except:
        age5 = 'x'
      man = s.select_one('#netizen_point_graph > div > div.grp_wrap > div.grp_gender > div:nth-of-type(1) > div > strong.graph_point')
      try:
        man =man.text
      except:
        man = 'x'
      woman = s.select_one('#netizen_point_graph > div > div.grp_wrap > div.grp_gender > div:nth-of-type(2) > div > strong.graph_point')
      try:
        woman = woman.text
      except:
        woman = 'x'
      title_list.append(title)
      star2_list.append(star2)
      age1_list.append(age1)
      age2_list.append(age2)
      age3_list.append(age3)
      age4_list.append(age4)
      age5_list.append(age5)
      man_list.append(man)
      woman_list.append(woman)
      star_list.append(star)
      jangru_list.append(jangru)
      country_list.append(country)
      time_list.append(time)
      date_list.append(date)
      info_list.append(inf)
    else:
      pass


In [ ]:
data = {'영화제목':title_list,
        '감독':gamdog_list,
        '등급':agerange_list,
        '관람객평점':star_list,
        '네티즌평점':star2_list,
        '장르':jangru_list,
        '국적':country_list,
        '영화시간':time_list,
        '개봉일':date_list,
        '추천':info_list,
        '10대 비율':age1_list,
        '20대 비율':age2_list,
        '30대 비율':age3_list,
        '40대 비율':age4_list,
        '50대 비율':age5_list,
        '남성 만족도':man_list,
        '여성 만족도':woman_list}
df = pd.DataFrame(data=data)
df = df.dropna()
df = df.reset_index(drop=True)
df

,영화제목,관람객평점,네티즌평점,장르,국적,영화시간,개봉일,추천,10대 비율,20대 비율,30대 비율,40대 비율,50대 비율,남성 만족도,여성 만족도
0,명량: 회오리 바다를 향하여,관람객 평점 6.41점,5.06,다큐멘터리,한국,95분,20150507 개봉,이 영화는 10대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,4%,19%,37%,37%,4%,4.06,6.67
1,명량,관람객 평점 8.88점,8.44,액션 드라마,한국,128분,20140730 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,7%,43%,25%,19%,5%,7.46,9.08
2,극한직업,관람객 평점 9.20점,8.50,코미디,한국,111분,20190123 개봉,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,2%,38%,30%,21%,8%,7.63,8.71
3,신과함께-죄와 벌,관람객 평점 8.73점,7.83,판타지 드라마,한국,139분,20210107 재개봉 20171220 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,5%,45%,25%,19%,5%,6.39,7.90
4,국제시장,관람객 평점 9.16점,9.02,드라마,한국,126분,20141217 개봉,이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,6%,40%,28%,18%,8%,8.74,9.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,안시성,관람객 평점 8.63점,7.99,액션,한국,135분,20180919 개봉,이 영화는 30대 여성이 좋아하는 연출이 뛰어난 영화입니다.,2%,34%,27%,27%,11%,6.72,8.92
141,블랙 팬서,관람객 평점 8.33점,7.51,액션 드라마,미국,135분,20180214 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,4%,54%,25%,14%,3%,6.61,7.90
142,더 킹: 헨리 5세,관람객 평점 8.39점,8.50,드라마,오스트레일리아 영국 헝가리,140분,20191023 개봉,이 영화는 30대 남성이 좋아하는 연기가 뛰어난 영화입니다.,1%,30%,33%,23%,12%,8.17,9.18
143,더 킹: 독립 전쟁,관람객 평점 없음,9.00,전쟁 액션 드라마,폴란드,107분,20210318 개봉,이 영화는 40대 남성이 좋아하는 연출이 뛰어난 영화입니다.,x,x,x,x,x,10.0,5.50


In [ ]:
df.to_csv("네이버_영화+비율_및_성별만족도.csv")

In [ ]:
df

,영화제목,관람객평점,네티즌평점,장르,국적,영화시간,개봉일,추천,10대 비율,20대 비율,30대 비율,40대 비율,50대 비율,남성 만족도,여성 만족도
0,명량: 회오리 바다를 향하여,관람객 평점 6.41점,5.06,다큐멘터리,한국,95분,20150507 개봉,이 영화는 10대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,4%,19%,37%,37%,4%,4.06,6.67
1,명량,관람객 평점 8.88점,8.44,액션 드라마,한국,128분,20140730 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,7%,43%,25%,19%,5%,7.46,9.08
2,극한직업,관람객 평점 9.20점,8.50,코미디,한국,111분,20190123 개봉,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,2%,38%,30%,21%,8%,7.63,8.71
3,신과함께-죄와 벌,관람객 평점 8.73점,7.83,판타지 드라마,한국,139분,20210107 재개봉 20171220 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,5%,45%,25%,19%,5%,6.39,7.90
4,국제시장,관람객 평점 9.16점,9.02,드라마,한국,126분,20141217 개봉,이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,6%,40%,28%,18%,8%,8.74,9.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
699,아내가 결혼했다,관람객 평점 없음,5.07,멜로로맨스 드라마,한국,119분,20081023 개봉,"이 영화는 20대 여성이 좋아하는 스토리, 연기가 뛰어난 영화입니다.",x,x,x,x,x,4.06,6.50
700,내 아내와 결혼해 주세요,관람객 평점 8.13점,7.78,멜로로맨스 드라마,일본,114분,20170518 개봉,이 영화는 30대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,0%,62%,25%,0%,12%,7.79,8.64
701,결혼하고도 싱글로 남는 법,관람객 평점 없음,8.52,코미디 멜로로맨스,프랑스,90분,20071206 개봉,이 영화는 20대 여성이 좋아하는 스토리가 뛰어난 영화입니다.,x,x,x,x,x,8.34,8.72
702,아저씨 우리 결혼할까요?,관람객 평점 없음,7.37,코미디 멜로로맨스,홍콩,103분,20041015 개봉,이 영화는 10대 남성이 좋아하는 영화입니다.,x,x,x,x,x,7.82,6.81


# 네이버 평점순 크롤링
 - 네이버 평점순 랭킹에 있는 데이터를 가져오겠습니다.

In [ ]:
title_list =[]
star_list = []
jangru_list = []
country_list = []
time_list =[]
date_list = []
info_list = []
age1_list = []
age2_list = []
age3_list = []
age4_list = []
age5_list = []
man_list = []
woman_list = []
star2_list = []
story_list =[]
gamdog_list =[]
agerange_list = []
date='20210929'    ## 
for page in tqdm(range(1,41)):
  search_url = 'https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=pnt&'
  url = f'{search_url}&date={date}&page={page}'
  req = requests.get(url)
  html = req.text
  soup =BeautifulSoup(html, 'html.parser')
  for i in range(1,56):
    if i == 1 or i == 12 or i == 23 or i == 34 or i == 45:
      pass
    else:
      tits = soup.select_one(f'#old_content > table > tbody > tr:nth-of-type({i}) > td.title > div > a')
      tits['href']
      new_name = tits['href']     # 영화 사이트 주소 가져오기
      new_url = f'https://movie.naver.com/{new_name}'
      new_url = new_url.replace('basic','point')      # 사이트에서 평점을 클릭 해야함
      new_req = requests.get(new_url)
      new_html = new_req.text
      s =BeautifulSoup(new_html, 'html.parser')
      
      title = s.select_one('#content > div.article > div.mv_info_area > div.mv_info > h3 > a')
      try:
        title = title.text     ### 영화 제목
      except:
        title = None
      gamdog = s.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd:nth-of-type(2)')
      try:
        gamdog = gamdog.text     ### 영화 감독
      except:
        gamdog = None
      ### 연령
      agerange = s.select_one('.info_spec')
      try:
        if '국내' in test(agerange.text.split('\n\n')[3]):
          y = test(agerange.text.split('\n\n')[3])
        else:
          y = test(agerange.text.split('\n\n')[4])
        y1 = y.find('국내')
        y2 = y.find('가')
        agerange = y[y1+2:y2+1]
      except:
        agerange = None

      inf_m = s.select_one("#netizen_point_graph > div > strong")
      try:
        inf = inf_m.text       ### 네이버 영화측에서 분석한 성별 연령층 정보 + 영화의 특징
      except:
        inf = None

      star = s.select_one("div.main_score div.score a div.star_score span.st_off span.st_on")
      try:
        star = star.text       ### 관람객 평점
      except:
        star = None

      star2 = s.select_one("#pointNetizenPersentBasic")
      try:
        star2 = star2.text      ### 네티즌 평점
      except:
        star2 = None

      date = s.select_one('#content > div.article > div.mv_info_area > div.mv_info > dl > dd > p')
      try:
        date = date.text
      except:
        date = None

      try:
        m = date.split('\n\n') ## \n\n으로 구분 돼 장르, 나라, 시간, 개봉일이 표시되는 것 같습니다.
      except:
        m = [None, None, None, None]

      try:
        jangru = test(m[1])
      except:
        jangru = None

      try:
        country = test(m[2])
      except:
        country = None

      try:
        time = test(m[3])
      except:
        time = None

      try:
        date = test(m[4])
      except:
        date = None

      age1 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(1) > strong.graph_percent")
      try:
        age1 = age1.text
      except:
        age1 = 'x'

      age2 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(2) > strong.graph_percent")
      try:
        age2 = age2.text
      except:
        age2 = 'x'

      age3 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(3) > strong.graph_percent")
      try:
        age3 = age3.text
      except:
        age3 = 'x' 

      age4 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(4) > strong.graph_percent")
      try:
        age4 = age4.text
      except:
        age4 = 'x'   

      age5 = s.select_one(f"#content > div.article > div.mv_info_area > div.mv_info > div.viewing_graph > div > div.bar_graph > div:nth-of-type(5) > strong.graph_percent")
      try:
        age5 = age5.text
      except:
        age5 = 'x'

      man = s.select_one('#netizen_point_graph > div > div.grp_wrap > div.grp_gender > div:nth-of-type(1) > div > strong.graph_point')
      try:
        man =man.text
      except:
        man = 'x'

      woman = s.select_one('#netizen_point_graph > div > div.grp_wrap > div.grp_gender > div:nth-of-type(2) > div > strong.graph_point')
      try:
        woman = woman.text
      except:
        woman = 'x'
      title_list.append(title)
      star2_list.append(star2)
      age1_list.append(age1)
      age2_list.append(age2)
      age3_list.append(age3)
      age4_list.append(age4)
      age5_list.append(age5)
      man_list.append(man)
      woman_list.append(woman)
      star_list.append(star)
      jangru_list.append(jangru)
      country_list.append(country)
      time_list.append(time)
      date_list.append(date)
      info_list.append(inf)
      gamdog_list.append(gamdog)
      agerange_list.append(agerange)

  0%|          | 0/40 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [ ]:
data = {'영화제목':title_list,
        '감독':gamdog_list,
        '등급':agerange_list,
        '관람객평점':star_list,
        '네티즌평점':star2_list,
        '장르':jangru_list,
        '국적':country_list,
        '영화시간':time_list,
        '개봉일':date_list,
        '추천':info_list,
        '10대 비율':age1_list,
        '20대 비율':age2_list,
        '30대 비율':age3_list,
        '40대 비율':age4_list,
        '50대 비율':age5_list,
        '남성 만족도':man_list,
        '여성 만족도':woman_list}
df = pd.DataFrame(data=data)
df.to_csv('네이버_랭킹데이터.csv')
df

# 텍스트 데이터 크롤링

In [ ]:
story_list=[]
act_list=[]
review_list=[]
for page in tqdm(range(1,41)):
  search_url = 'https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=pnt&'
  url = f'{search_url}&date={date}&page={page}'
  req = requests.get(url)
  html = req.text
  soup =BeautifulSoup(html, 'html.parser')
  for i in range(1,56):
    if i == 1 or i == 12 or i == 23 or i == 34 or i == 45:
      pass
    else:
      tits = soup.select_one(f'#old_content > table > tbody > tr:nth-of-type({i}) > td.title > div > a')
      tits['href']
      new_name = tits['href']     # 영화 사이트 주소 가져오기
      new_url = f'https://movie.naver.com/{new_name}'
      new_req = requests.get(new_url)
      new_html = new_req.text
      ss =BeautifulSoup(new_html, 'html.parser')        ## 줄거리

      new_url = new_url.replace('basic','point')      # 사이트에서 평점을 클릭 해야함
      new_req = requests.get(new_url)
      new_html = new_req.text
      s =BeautifulSoup(new_html, 'html.parser')       # 평점

      new_url = new_url.replace('basic','detail')      # 사이트에서 평점을 클릭 해야함
      new_req = requests.get(new_url)
      new_html = new_req.text
      sss =BeautifulSoup(new_html, 'html.parser')       ### 영화 제작 배우
      ### 줄거리
      story=s.select_one('#content > div.article > div.section_group.section_group_frst > div:nth-of-type(1) > div > div.story_area > h5')
      story2 = s.select_one('#content > div.article > div.section_group.section_group_frst > div:nth-of-type(1) > div > div.story_area > p')
      try:
        story = story.text.replace('\n','. ')
        story2 = story2.text.replace('\xa0',' ')
        story = story + '  -  ' + story2
      except:
        story = None

      act=[]            #### 배우 최대 8명까지
      for i in range(1,9):
        a = sss.select_one(f'#content > div.article > div.section_group.section_group_frst > div.obj_section.noline > div > div.lst_people_area.height100 > ul > li:nth-of-type({i}) > div > a')
      try:
        act.append(a['title'])
      except:
        pass

      story_list.append(story)
      act_list.append(act)


      

In [ ]:


url = f'https://movie.naver.com/movie/bi/mi/basic.naver?code=186114'
req = requests.get(url)
html = req.text
soup =BeautifulSoup(html, 'html.parser')
tits = soup.select_one(f'#old_content > table > tbody > tr:nth-of-type(2) > td.title > div > a')
tits['href']
new_name = tits['href']     # 영화 사이트 주소 가져오기
new_url = f'https://movie.naver.com/{new_name}'
#new_url = new_url.replace('basic','point')      # 사이트에서 평점을 클릭 해야함
new_req = requests.get(new_url)
new_html = new_req.text
s =BeautifulSoup(new_html, 'html.parser')
new_url = new_url.replace('basic','detail')      # 사이트에서 평점을 클릭 해야함
new_req = requests.get(new_url)
new_html = new_req.text
sss =BeautifulSoup(new_html, 'html.parser')   

In [ ]:
wd.get(new_url)

In [ ]:

new_url = f'https://movie.naver.com/{new_name}'
new_req = requests.get(new_url)
new_html = new_req.text
ss =BeautifulSoup(new_html, 'html.parser')        ## 줄거리

new_url = new_url.replace('basic','point')      # 사이트에서 평점을 클릭 해야함
new_req = requests.get(new_url)
new_html = new_req.text
s =BeautifulSoup(new_html, 'html.parser')       # 평점

new_url = new_url.replace('basic','detail')      # 사이트에서 평점을 클릭 해야함
new_req = requests.get(new_url)
new_html = new_req.text
sss =BeautifulSoup(new_html, 'html.parser')       ### 영화 제작 배우

In [ ]:
re = s.select('body > div')
re

In [ ]:
new_url = 'https://movie.naver.com/movie/bi/mi/detail.naver?code=173247'
new_req = requests.get(new_url)
new_html = new_req.text
sss =BeautifulSoup(new_html, 'html.parser')

In [ ]:
act = sss.select_one('#content > div.article > div.section_group.section_group_frst > div:nth-of-type(1) > div > div > h5')
act

In [ ]:
df = pd.read_csv('네이버_랭킹1_2000.csv')
df2 = pd.read_csv('kobis_upto_2021-09-28.csv')

In [ ]:
df2 =df2.drop(df2.index[0]).reset_index()
del df2['index']
df2.head(1)

,순번,영화명,감독,제작사,수입사,배급사,개봉일,영화유형,영화형태,국적,전국,전국.1,전국.2,서울,서울.1,장르,등급,영화구분
0,1.0,명량,김한민,(주)빅스톤픽쳐스,NaN,(주)씨제이이엔엠,2014-07-30,개봉영화,장편,한국,"1,587","135,748,398,910","17,613,682","33,121,225,810","4,163,666",사극,15세이상관람가,일반영화


In [ ]:
df3 = pd.read_csv('ever_kobis.csv')
df3

,순위,영화명,개봉일,매출액,관객수,스크린수,국적,배급사
0,1.0,명량,2014-07-30,1.357484e+11,17613682.0,1587.0,한국,(주)씨제이이엔엠
1,2.0,극한직업,2019-01-23,1.396480e+11,16264944.0,1978.0,한국,(주)씨제이이엔엠
2,3.0,신과함께-죄와 벌,2017-12-20,1.156987e+11,14410754.0,1912.0,한국,롯데쇼핑㈜롯데엔터테인먼트
3,4.0,국제시장,2014-12-17,1.109135e+11,14257115.0,966.0,한국,(주)씨제이이엔엠
4,5.0,어벤져스: 엔드게임,2019-04-24,1.221827e+11,13934592.0,2835.0,미국,월트디즈니컴퍼니코리아 유한책임회사
...,...,...,...,...,...,...,...,...
495,496.0,목포는 항구다,2004-02-20,0.000000e+00,1795700.0,32.0,한국,코리아픽쳐스(주)
496,497.0,강철비2: 정상회담,2020-07-29,1.465537e+10,1790797.0,2137.0,한국,롯데컬처웍스(주)롯데엔터테인먼트
497,498.0,분노의 질주: 더 맥시멈,2013-05-22,1.320352e+10,1790457.0,656.0,미국,유니버설픽쳐스인터내셔널 코리아(유)
498,499.0,아내가 결혼했다,2008-10-23,1.199586e+10,1788748.0,414.0,한국,CJ ENM


In [ ]:
df3 = df2[['순번','영화명','개봉일','전국.1','전국.2','전국','국적','배급사']]
df3 = df3.rename(columns={'순번':'순위'})
df3 = df3.rename(columns={'전국.1':'매출액'})
df3 = df3.rename(columns={'전국.2':'관객수'})
df3 = df3.rename(columns={'전국':'스크린수'})

In [ ]:
#df3 = df3.rename(columns={'영화명':'영화제목'})
df3.to_csv('10월1일KOBIS데이터.csv')

In [ ]:
df4 = pd.merge(df3,df6,how='inner', on = '영화명')

In [ ]:
df4.head(1)
del df4['개봉일_x']
del df4['배급사']
del df4['영화시간']
del df4['국적_x']
df4.head(1)

,순위,영화명,매출액,관객수,스크린수,감독,등급,관람객평점,네티즌평점,장르,국적_y,개봉일_y,추천,10대 비율,20대 비율,30대 비율,40대 비율,50대 비율,남성 만족도,여성 만족도
0,1.0,명량,"135,748,398,910","17,613,682","1,587",김한민,15세 관람가,관람객 평점 8.88점,8.44,액션 드라마,한국,20140730 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,7%,43%,25%,19%,5%,7.46,9.08


In [ ]:
df4 = df4.rename(columns={'국적_y':'국적'})
df4 = df4.rename(columns={'개봉일_y':'개봉일'})

In [ ]:
del df5['Unnamed: 0']

In [ ]:
df10 = pd.concat([df4,df5],ignore_index=True)
df10 = df10.drop_duplicates(['영화명']).reset_index()

In [ ]:

del df10['level_0']

In [ ]:
df10.to_csv

,순위,영화명,매출액,관객수,스크린수,감독,등급,관람객평점,네티즌평점,장르,국적,개봉일,추천,10대 비율,20대 비율,30대 비율,40대 비율,50대 비율,남성 만족도,여성 만족도
0,1.0,명량,"135,748,398,910","17,613,682","1,587",김한민,15세 관람가,관람객 평점 8.88점,8.44,액션 드라마,한국,20140730 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,7%,43%,25%,19%,5%,7.46,9.08
1,2.0,극한직업,"139,647,979,516","16,264,944","1,978",이병헌,15세 관람가,관람객 평점 9.20점,8.50,코미디,한국,20190123 개봉,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,2%,38%,30%,21%,8%,7.63,8.71
2,3.0,신과함께-죄와 벌,1.15699e+11,1.44108e+07,1912,김용화,12세 관람가,관람객 평점 8.73점,7.83,판타지 드라마,한국,20210107 재개봉 20171220 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,5%,45%,25%,19%,5%,6.39,7.9
3,4.0,국제시장,"110,828,014,630","14,245,998",966,윤제균,12세 관람가,관람객 평점 9.16점,9.02,드라마,한국,20141217 개봉,이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,6%,40%,28%,18%,8%,8.74,9.27
4,5.0,어벤져스: 엔드게임,"122,182,694,160","13,934,592","2,835","안소니 루소, 조 루소",12세 관람가,관람객 평점 9.49점,9.38,액션,미국,20190424 개봉,이 영화는 20대 남성이 좋아하는 연출이 뛰어난 영화입니다.,4%,46%,29%,17%,3%,9.20,9.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,223.0,아이 캔 스피크,"25,532,214,986","3,280,243","1,140",김현석,12세 관람가,관람객 평점 9.23점,9.37,드라마,한국,20170921 개봉,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,6%,52%,21%,15%,6%,9.13,9.71
196,225.0,스타워즈: 깨어난 포스,2.83538e+10,3.27388e+06,940,J.J. 에이브럼스,12세 관람가,관람객 평점 8.21점,7.74,액션 모험 판타지,미국,20151217 개봉,이 영화는 20대 남성이 좋아하는 영상미가 뛰어난 영화입니다.,3%,31%,30%,31%,5%,7.17,8.07
197,225.0,올드보이,0,"3,269,000",0,박찬욱,청소년 관람불가,관람객 평점 10.00점,9.04,드라마 미스터리 범죄 스릴러,한국,20131121 재개봉 20031121 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,0%,50%,50%,0%,0%,9.13,8.91
198,226.0,분노의 질주: 더 세븐,"26,449,334,694","3,247,558",983,제임스 완,15세 관람가,관람객 평점 9.21점,9.12,액션 범죄 스릴러,미국,20150401 개봉,이 영화는 20대 남성이 좋아하는 연출이 뛰어난 영화입니다.,3%,54%,29%,11%,3%,8.97,9.47


In [ ]:
df5=pd.read_csv('500위데이터.csv')


,Unnamed: 0,순위,영화명,매출액,관객수,스크린수,감독,등급,관람객평점,네티즌평점,장르,국적,개봉일,추천,10대 비율,20대 비율,30대 비율,40대 비율,50대 비율,남성 만족도,여성 만족도
0,0,1.0,명량,1.357484e+11,17613682.0,1587.0,김한민,15세 관람가,관람객 평점 8.88점,8.44,액션 드라마,한국,20140730 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,7%,43%,25%,19%,5%,7.46,9.08


In [ ]:
df6 = pd.read_csv('네이버2000(10월1일).csv')
del df6['Unnamed: 0']
del df6['Unnamed: 0.1']

In [ ]:
df6
df6 = df6.rename(columns={'영화제목':'영화명'})

In [ ]:
df4

,순위,영화명,매출액,관객수,스크린수,감독,등급,관람객평점,네티즌평점,장르,국적,개봉일,추천,10대 비율,20대 비율,30대 비율,40대 비율,50대 비율,남성 만족도,여성 만족도
0,1.0,명량,"135,748,398,910","17,613,682","1,587",김한민,15세 관람가,관람객 평점 8.88점,8.44,액션 드라마,한국,20140730 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,7%,43%,25%,19%,5%,7.46,9.08
1,2.0,극한직업,"139,647,979,516","16,264,944","1,978",이병헌,15세 관람가,관람객 평점 9.20점,8.50,코미디,한국,20190123 개봉,이 영화는 20대 여성이 좋아하는 연기가 뛰어난 영화입니다.,2%,38%,30%,21%,8%,7.63,8.71
2,4.0,국제시장,"110,828,014,630","14,245,998",966,윤제균,12세 관람가,관람객 평점 9.16점,9.02,드라마,한국,20141217 개봉,이 영화는 20대 남성이 좋아하는 스토리가 뛰어난 영화입니다.,6%,40%,28%,18%,8%,8.74,9.27
3,5.0,어벤져스: 엔드게임,"122,182,694,160","13,934,592","2,835","안소니 루소, 조 루소",12세 관람가,관람객 평점 9.49점,9.38,액션,미국,20190424 개봉,이 영화는 20대 남성이 좋아하는 연출이 뛰어난 영화입니다.,4%,46%,29%,17%,3%,9.20,9.50
4,6.0,겨울왕국 2,"114,810,421,450","13,747,792","2,648","크리스 벅, 제니퍼 리",전체 관람가,관람객 평점 8.95점,8.70,애니메이션 모험 코미디 가족 판타지 뮤지컬,미국,20191121 개봉,이 영화는 20대 여성이 좋아하는 영상미가 뛰어난 영화입니다.,4%,32%,35%,25%,4%,8.26,8.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1590,18035.0,헤드윅,0,0,0,존 카메론 미첼,15세 관람가,관람객 평점 9.46점,9.19,드라마 코미디 뮤지컬,미국,20170628 재개봉 20020809 개봉,이 영화는 20대 여성이 좋아하는 OST가 뛰어난 영화입니다.,5%,68%,20%,6%,1%,8.64,9.51
1591,18041.0,혹성탈출,0,0,0,프랭클린 J. 샤프너,15세 관람가,관람객 평점 없음,9.23,액션 드라마 미스터리,미국,19690925 개봉,이 영화는 10대 남성이 좋아하는 연출이 뛰어난 영화입니다.,x,x,x,x,x,9.15,9.64
1592,18044.0,화양연화,0,0,0,왕가위,15세 관람가,관람객 평점 9.17점,8.77,드라마 멜로로맨스,홍콩 프랑스,20201224 재개봉 20131128 재개봉 20001021 개봉,이 영화는 20대 여성이 좋아하는 영상미가 뛰어난 영화입니다.,3%,46%,33%,7%,10%,8.47,8.96
1593,18058.0,희극지왕,0,0,0,"주성치(사우), 막문위(부망), 장바이즈(피우)더보기",12세 관람가,관람객 평점 없음,8.86,코미디 액션,홍콩,20000226 개봉,이 영화는 20대 남성이 좋아하는 연기가 뛰어난 영화입니다.,x,x,x,x,x,8.80,9.20


In [ ]:
df4.shape , df10.shape

((1595, 20), (1693, 20))

In [ ]:
a = df4['영화명'].values.tolist()
b = df10['영화명'].values.tolist()

In [ ]:
df4[df4['영화명']=='라스트 미션']

,순위,영화명,매출액,관객수,스크린수,감독,등급,관람객평점,네티즌평점,장르,국적,개봉일,추천,10대 비율,20대 비율,30대 비율,40대 비율,50대 비율,남성 만족도,여성 만족도
865,2443.0,라스트 미션,"835,560,639","101,293",472,클린트 이스트우드,15세 관람가,관람객 평점 8.91점,NaN,드라마,미국,20190314 개봉,이 영화는 40대 남성이 좋아하는 연기가 뛰어난 영화입니다.,1%,28%,30%,18%,23%,8.85,8.95
866,10849.0,라스트 미션,"32,000",28,5,클린트 이스트우드,15세 관람가,관람객 평점 8.91점,NaN,드라마,미국,20190314 개봉,이 영화는 40대 남성이 좋아하는 연기가 뛰어난 영화입니다.,1%,28%,30%,18%,23%,8.85,8.95


In [ ]:
li=[]
for i in range(len(df10['순위'])):
  if df10['순위'][i] != None:
    a = None
    li.append(a)


In [ ]:
li2=[]
for i in range(len(df4['순위'])):
  if df10['순위'][i] != None:
    a = 1
    li2.append(a)

In [ ]:
del df10['Na']

In [ ]:
df11 = pd.concat([df10,df4['Na']],axis=1)

In [ ]:
a = df11[df11['Na'].isna()]

In [ ]:
a.to_csv('추가된_영화명(블로그,한줄평필요).csv')